In [ ]:
#Import libraries for HMM
import jax.numpy as jnp
import jax.random as jr

from itertools import count
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("notebook")
from sklearn.metrics import r2_score
import scipy
from dynamax.hidden_markov_model import LinearRegressionHMM
from dynamax.utils.plotting import CMAP, COLORS, white_to_color_cmap
#Import data handling of monkey pac-man
import PacTimeOrig.DataHandling as DH
import PacTimeOrig.Methods.utils as pacutils

#Import standard libraries
import numpy as np
import pandas as pd
import os
from scipy.io import loadmat
from scipy.io import savemat

In [ ]:
datfolder='/Users/user/PycharmProjects/PacManMain/data/HumanHealthy/Original/Mat Files/behavior'

In [ ]:
#enumerate subject file
for _,fil in enumerate(os.listdir(datfolder)):
    tmpdir={}
    dat=loadmat(datfolder+'/'+fil)
    tmpdir['data']=dat['data']
    tmpdir['ntrials']=tmpdir['data'].shape[1]

    for trial in range(tmpdir['ntrials']):
        #retain only 2 prey trials for this HMM-switching
        if pd.DataFrame(tmpdir['data'][0][trial][0]).numPrey[0][0][0]==2:
            sjX=np.array(pd.DataFrame(tmpdir['data'][0][trial][0]).subj_xPath[0][0]).reshape(-1,1)
            sjY=np.array(pd.DataFrame(tmpdir['data'][0][trial][0]).subj_yPath[0][0]).reshape(-1,1)
            sjpos=np.hstack((sjX,sjY)).astype('float32')
            #prey 1
            aX=np.array(pd.DataFrame(tmpdir['data'][0][trial][0]).NPC_xPath[0][0,:]).reshape(-1,1)
            #prey 2
            bX=np.array(pd.DataFrame(tmpdir['data'][0][trial][0]).NPC_xPath[0][1,:]).reshape(-1,1)
            #prey 1
            aY=np.array(pd.DataFrame(tmpdir['data'][0][trial][0]).NPC_yPath[0][0,:]).reshape(-1,1)
            #prey 2
            bY=np.array(pd.DataFrame(tmpdir['data'][0][trial][0]).NPC_yPath[0][1,:]).reshape(-1,1)

            A=np.hstack((aX,aY))
            B=np.hstack((bX,bY))


            RT=np.where((vel.__pow__(2).sum(axis=1))>(0.05*np.max(vel.__pow__(2).sum(axis=1))))[0][0]
            A=A[RT:,:]
            B=B[RT:,:]
            sjpos=sjpos[RT:,:]
            erA = A - sjpos
            erB = B - sjpos


            vel = np.array([np.gradient(sjpos[:, 0], 1), np.gradient(sjpos[:, 1], 1)]).transpose()


            erA = jnp.array(erA)
            erB = jnp.array(erB)

            vel = jnp.array(vel)
            inputs = np.hstack([erA, erB])
            keys=jr.PRNGKey(np.random.randint(1,100+1))
            #keys = map(jr.PRNGKey, count())
            hmm = LinearRegressionHMM(2, 4, 2)
            #test_params, param_props = hmm.initialize(next(keys))
            test_params, param_props = hmm.initialize(keys)

            test_params, lps = hmm.fit_em(test_params, param_props, vel, inputs=inputs,num_iters=300,verbose=False)

            most_likely_states = hmm.most_likely_states(test_params, vel, inputs=inputs)

            stateposterior = hmm.filter(test_params, vel, inputs=inputs)
            smoothposterior = hmm.smoother(test_params,vel,inputs=inputs)
            rat=np.zeros(test_params.emissions.weights.shape[0])
            for k in range(test_params.emissions.weights.shape[0]):
                rat[k]=np.sqrt(test_params.emissions.weights[k][0:2,0:2].flatten().__pow__(2).sum())/np.sqrt(test_params.emissions.weights[k][0:2,3:4].flatten().__pow__(2).sum())


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
#import torch_optimizer as optim
import numpy as np
import control as ctl
from scipy.io import loadmat
from scipy.io import savemat
from tqdm import tqdm

import matplotlib.pyplot as plt

from PacTimeOrig.Methods import ModelFitting as mf




sjX=np.array(pd.DataFrame(tmpdir['data'][0][trial][0]).subj_xPath[0][0]).reshape(-1,1)
sjY=np.array(pd.DataFrame(tmpdir['data'][0][trial][0]).subj_yPath[0][0]).reshape(-1,1)
y=torch.tensor(np.hstack((sjX,sjY)).astype('float32'))
#prey 1
aX=np.array(pd.DataFrame(tmpdir['data'][0][trial][0]).NPC_xPath[0][0,:]).reshape(-1,1)
#prey 2
bX=np.array(pd.DataFrame(tmpdir['data'][0][trial][0]).NPC_xPath[0][1,:]).reshape(-1,1)
#prey 1
aY=np.array(pd.DataFrame(tmpdir['data'][0][trial][0]).NPC_yPath[0][0,:]).reshape(-1,1)
#prey 2
bY=np.array(pd.DataFrame(tmpdir['data'][0][trial][0]).NPC_yPath[0][1,:]).reshape(-1,1)

A=np.hstack((aX,aY))
B=np.hstack((bX,bY))

A=torch.tensor(A.astype('float32'))
B=torch.tensor(B.astype('float32'))



init_y = (y[0, :])

#Uncomment to provide the inputs
#model = mf.LDSMIXTURELQR(init_y,A,B,input_dim=1,r1=R1,r2=R2)

model = mf.LDSMIXTURELQR(init_y,A,B,input_dim=1,startW=torch.tensor([0]))


criterion = nn.MSELoss()
#optimizer = optim.Adahessian(model.parameters(), lr=0.01,hessian_power=0.1)
optimizer = optim.Adam(model.parameters(), lr=0.01)
# Prepare the dataset

# Training loop
for epoch in tqdm(range(2000)):
    optimizer.zero_grad()
    x, yprime, u = model()
    loss = criterion(yprime[1:-1, :], y[1,:])
    loss = loss + (0.00000000001 * (torch.abs(torch.diff(x)).mean()))
    loss.backward(create_graph=True)
    #torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
    optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
#import torch_optimizer as optim
import numpy as np
import control as ctl
from scipy.io import loadmat
from scipy.io import savemat
from tqdm import tqdm

import matplotlib.pyplot as plt

from PacTimeOrig.Methods import ModelFitting as mf


sjX=np.array(pd.DataFrame(tmpdir['data'][0][trial][0]).subj_xPath[0][0]).reshape(-1,1)
sjY=np.array(pd.DataFrame(tmpdir['data'][0][trial][0]).subj_yPath[0][0]).reshape(-1,1)
y=(np.hstack((sjX,sjY)).astype('float32'))
#prey 1
aX=np.array(pd.DataFrame(tmpdir['data'][0][trial][0]).NPC_xPath[0][0,:]).reshape(-1,1)
#prey 2
bX=np.array(pd.DataFrame(tmpdir['data'][0][trial][0]).NPC_xPath[0][1,:]).reshape(-1,1)
#prey 1
aY=np.array(pd.DataFrame(tmpdir['data'][0][trial][0]).NPC_yPath[0][0,:]).reshape(-1,1)
#prey 2
bY=np.array(pd.DataFrame(tmpdir['data'][0][trial][0]).NPC_yPath[0][1,:]).reshape(-1,1)

A=np.hstack((aX,aY))
B=np.hstack((bX,bY))
vel = np.array([np.gradient(y[:, 0], 1), np.gradient(y[:, 1], 1)]).transpose()

RT=np.where((vel.__pow__(2).sum(axis=1))>(0.05*np.max(vel.__pow__(2).sum(axis=1))))[0][0]
A=A[RT:,:]
B=B[RT:,:]
y=y[RT:,:]


y=torch.tensor(y)
A=torch.tensor(A.astype('float32'))
B=torch.tensor(B.astype('float32'))



init_y = (y[0, :])

nRBF=41
model = mf.RBFMIXTURELQR(y, A, B, A_prime=1, B_prime=1 / 60, nRBF=nRBF)

criterion=nn.MSELoss()
#optimizer = optim.Adahessian(model.parameters(), lr=0.01,hessian_power=0.4)
optimizer = optim.Adam(model.parameters(), lr=0.01)


# %compute smoothness constraint in weights
LAP=2*torch.eye(nRBF)
LAP[0,0]=1.0
LAP[-1,-1]=1.0

matrix = torch.zeros(nRBF, nRBF)

# Fill in the superdiagonal with -1s
rows, cols = torch.diag(torch.full((nRBF - 1,), -1), diagonal=1).nonzero(as_tuple=True)
matrix[rows, cols] = -1

# Fill in the subdiagonal with -1s
rows, cols = torch.diag(torch.full((nRBF - 1,), -1), diagonal=-1).nonzero(as_tuple=True)
matrix[rows, cols] = -1

LAP=LAP+matrix


for epoch in (range(5000)):
    optimizer.zero_grad()
    y_pred, u_pred = model()
    loss = criterion(y_pred[1:, :], y[1:,:])
    loss=loss+0.00001*(model.weights.T @ LAP @ model.weights)
    #loss = loss + (0.00001 * (torch.abs(torch.diff(x)).mean()))
    loss.backward(create_graph=True)
    #torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
    optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')